In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install ultralytics pyttsx3  # Install YOLOv8 + TTS
!sudo apt-get install espeak      # Required for pyttsx3 (Linux backend)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.0/950.0 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
%cd "/content/drive/MyDrive/Artificial Intelligence Studio"

/content/drive/MyDrive/Artificial Intelligence Studio


In [2]:
import os
import yaml

# Path to your dataset directory (parent of train/valid folders)
dataset_path = "/content/drive/MyDrive/Artificial Intelligence Studio/Detect.v2i.yolov8"

# Path to your labels
label_dir = os.path.join(dataset_path, "train/labels")

# Get all unique class IDs from label files
class_ids = set()
for label_file in os.listdir(label_dir):
    if label_file.endswith(".txt"):
        with open(os.path.join(label_dir, label_file), 'r') as f:
            for line in f.readlines():
                if line.strip():  # Skip empty lines
                    try:
                        class_id = int(line.split()[0])  # First value is class_id
                        class_ids.add(class_id)
                    except (IndexError, ValueError):
                        continue

# Sort class IDs and assign names (you can customize names later)
class_names = {i: f"class_{i}" for i in sorted(class_ids)}  # Default names: class_0, class_1, etc.

# Create data.yaml
data_yaml = {
    "path": dataset_path,
    "train": "train/images",
    "val": "valid/images",
    "names": class_names
}

# Save to file
output_path = os.path.join(dataset_path, "data.yaml")
with open(output_path, 'w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print(f"Generated data.yaml at {output_path} with {len(class_names)} classes:")
print(class_names)

Generated data.yaml at /content/drive/MyDrive/Artificial Intelligence Studio/Detect.v2i.yolov8/data.yaml with 6 classes:
{0: 'class_0', 1: 'class_1', 2: 'class_2', 3: 'class_3', 4: 'class_4', 5: 'class_5'}


In [6]:
from ultralytics import YOLO

# Load model (YOLOv8 Nano is lightest for CPU)
model = YOLO("yolov8n.pt")

# Train on CPU
results = model.train(
    data="/content/drive/MyDrive/Artificial Intelligence Studio/Detect.v2i.yolov8/data.yaml",  # Your dataset YAML
    epochs=50,
    batch=8,              # Reduce batch size (CPU memory is limited)
    imgsz=320,            # Smaller image size = faster on CPU
    device="cpu",         # Force CPU training
    name="yolov8_cpu"     # Save results to 'runs/detect/yolov8_cpu'
)

100%|██████████| 6.25M/6.25M [00:00<00:00, 163MB/s]


Ultralytics 8.3.98 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Artificial Intelligence Studio/Detect.v2i.yolov8/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=320, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=yolov8_cpu, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

100%|██████████| 755k/755k [00:00<00:00, 75.5MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/Artificial Intelligence Studio/Detect.v2i.yolov8/train/labels.cache... 88 images, 0 backgrounds, 0 corrupt: 100%|██████████| 88/88 [00:00<?, ?it/s]


...

      50/50         0G     0.6045     0.8411      1.007         23        320: 100%|██████████| 11/11 [00:19<00:00,  1.77s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.11s/it]

                   all         25         62      0.619      0.715      0.734      0.568



50 epochs completed in 0.313 hours.
Optimizer stripped from runs/detect/yolov8_cpu/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8_cpu/weights/best.pt, 6.2MB

Validating runs/detect/yolov8_cpu/weights/best.pt...
Ultralytics 8.3.98 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.15s/it]


                   all         25         62      0.621      0.715      0.734      0.568
               class_0         25         30      0.849      0.967      0.984      0.828
               class_2          6          8      0.714      0.625      0.639      0.494
               class_3          1          2      0.389        0.5      0.695       0.52
               class_4         14         20      0.861       0.85      0.954      0.671
               class_5          2          2      0.289      0.631      0.398      0.328
Speed: 2.2ms preprocess, 68.4ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/yolov8_cpu


In [12]:
from google.colab import files
from IPython.display import Image

# 1. List all evaluation files
!ls -la /content/runs/detect/yolov8_cpu2/

# 2. View confusion matrix
Image(filename='/content/runs/detect/yolov8_cpu2/confusion_matrix.png')

# 3. Download specific files
files.download('/content/runs/detect/yolov8_cpu2/results.csv')

total 4084
drwxr-xr-x 3 root root   4096 Mar 30 02:11 .
drwxr-xr-x 4 root root   4096 Mar 30 01:49 ..
-rw-r--r-- 1 root root   1608 Mar 30 01:49 args.yaml
-rw-r--r-- 1 root root 136261 Mar 30 02:11 confusion_matrix_normalized.png
-rw-r--r-- 1 root root 111660 Mar 30 02:11 confusion_matrix.png
-rw-r--r-- 1 root root 218892 Mar 30 02:11 events.out.tfevents.1743299398.acdc8b02b672.5429.0
-rw-r--r-- 1 root root 228473 Mar 30 02:11 F1_curve.png
-rw-r--r-- 1 root root 235217 Mar 30 01:50 labels_correlogram.jpg
-rw-r--r-- 1 root root 234433 Mar 30 01:50 labels.jpg
-rw-r--r-- 1 root root 202196 Mar 30 02:11 P_curve.png
-rw-r--r-- 1 root root 115616 Mar 30 02:11 PR_curve.png
-rw-r--r-- 1 root root 180078 Mar 30 02:11 R_curve.png
-rw-r--r-- 1 root root   6165 Mar 30 02:11 results.csv
-rw-r--r-- 1 root root 305219 Mar 30 02:11 results.png
-rw-r--r-- 1 root root 177243 Mar 30 01:51 train_batch0.jpg
-rw-r--r-- 1 root root 156904 Mar 30 01:51 train_batch1.jpg
-rw-r--r-- 1 root root 162671 Mar 30 01:

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
from ultralytics import YOLO

# Load your best model (automatically saved in runs/detect/)
model = YOLO('runs/detect/yolov8_cpu2/weights/best.pt')

metrics = model.val(
    data="/content/drive/MyDrive/Artificial Intelligence Studio/Detect.v2i.yolov8/data.yaml"
,
    batch=8,          # Match your training batch size
    imgsz=320,        # Match your training image size
    device='cpu',     # Use 'cuda' if GPU available
    split='val'       # Uses the 'val' set from data.yaml
)

Ultralytics 8.3.98 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/Artificial Intelligence Studio/Detect.v2i.yolov8/valid/labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.43it/s]


                   all         25         62      0.621      0.715      0.734      0.568
               class_0         25         30      0.849      0.967      0.984      0.828
               class_2          6          8      0.714      0.625      0.639      0.494
               class_3          1          2      0.389        0.5      0.695       0.52
               class_4         14         20      0.861       0.85      0.954      0.671
               class_5          2          2      0.289      0.631      0.398      0.328
Speed: 0.8ms preprocess, 81.8ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/val3


In [18]:
from collections import Counter
import glob

# Count instances per class in training set
label_files = glob.glob("/content/drive/MyDrive/Artificial Intelligence Studio/Detect.v2i.yolov8/train/labels*.txt")
class_counts = Counter()

for file in label_files:
    with open(file) as f:
        for line in f:
            class_id = int(line.split()[0])
            class_counts[class_id] += 1

print("Training set class distribution:", class_counts)

Training set class distribution: Counter()


In [19]:
from ultralytics import YOLO
from IPython.display import HTML
from base64 import b64encode

# Load your trained model
model = YOLO('runs/detect/yolov8_cpu2/weights/best.pt')

# Run detection on video
video_path = model.predict(
    source="https://www.youtube.com/watch?v=3-276V_mfJM",  # or YouTube URL
    conf=0.5,                 # Confidence threshold
    save=True,                # Save output video
    device="cpu",             # Use 'cuda' if GPU available
    show_labels=True,         # Display class names
    show_conf=True            # Display confidence scores
)

# Display results in Colab
output_path = video_path[0].save_dir + "/" + video_path[0].path
HTML(f"""
<video width=640 controls>
  <source src="{output_path}" type="video/mp4">
</video>
""")


requirements: Ultralytics requirement ['pytubefix>=6.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.5/731.5 kB 17.2 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 4.3s, installed 1 package: ['pytubefix>=6.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

1/1: https://www.youtube.com/watch?v=3-276V_mfJM... Success ✅ (153943 frames of shape 1920x1080 at 60.00 FPS)


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 192x

KeyboardInterrupt: 

In [5]:
from ultralytics import YOLO
from IPython.display import HTML, display
from base64 import b64encode
import os

# Load your trained model
model = YOLO('runs/detect/yolov8_cpu2/weights/best.pt')

# Run detection on YouTube video
results = model.predict(
    source= "/content/drive/MyDrive/Artificial Intelligence Studio/Detect.v2i.yolov8/2025-03-30 13-31-41.mp4",
    conf=0.5,
    save=True,
    device="cpu",
    stream=True,  # Critical for YouTube videos
    show_labels=True,
    show_conf=True,
    imgsz=320,
    project="youtube_results"  # Explicit output directory
)

# Process the generator to ensure completion
for result in results:
    pass  # This ensures all frames are processed

# Get the output video path (now accessible after processing)
output_dir = "youtube_results/predict"
output_files = os.listdir(output_dir)
output_video_path = f"{output_dir}/{output_files[0]}" if output_files else None

if output_video_path and os.path.exists(output_video_path):
    # Read the video file
    video_file = open(output_video_path, 'rb').read()

    # Create HTML video element
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    display(HTML(f"""
    <video width=800 controls autoplay loop>
        <source src="{video_url}" type="video/mp4">
    </video>
    """))

    # Download option
    from google.colab import files
    files.download(output_video_path)
else:
    print("Error: Output video not found. Check these paths:")
    print(f"Expected at: {output_dir}")
    !ls -la youtube_results/predict/  # Debug directory contents

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect/yolov8_cpu2/weights/best.pt'